### Import libraries

In [86]:
import pandas as pd

import camelot

import pdfplumber

import os

import re

import numpy as np

### Define Function to scroll through each file in the folder

In [70]:
def process_all_pdfs_in_folder(source_path, sink_path, desired_disease):
    year_df = []
    for filename in os.listdir(source_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(source_path, filename)
            print(f"Processando o arquivo: {pdf_path}")
            week_df = extract_and_process_tables(pdf_path, desired_disease, filename)
            year_df.append(week_df)
            
    if year_df:
        concatenated_df = pd.concat(year_df, ignore_index=True)

        pdf_filename = source_path.rsplit('/', 1)[-1]
        print("Salvando")
        csv_filename = os.path.splitext(pdf_filename)[0] + desired_disease + '.csv'
        output_path = os.path.join(sink_path, csv_filename)
        concatenated_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Arquivo salvo: {csv_filename}")
            

### Define function to scroll through each page/table in selected file

In [71]:
def extract_and_process_tables(pdf_path, desired_disease, filename):
    all_dfs = []
    pdf = pdfplumber.open(pdf_path)

    for page_num in range(len(pdf.pages)):
        try:
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

            if tables:
                df = tables[0].df
                page = pdf.pages[page_num]
                table = page.extract_table()
                df_header = pd.DataFrame(table)
                                
                if (not df_header.empty 
                    and df_header.shape[0] > 1
                    and df_header.shape[1] > 1
                    and df_header.iloc[0, 1] is not None 
                    and isinstance(df_header.iloc[0, 1], str)
                    and desired_disease in df_header.iloc[0, 1]):  
                    
                    first = None
                    

                    for n in range(4, 10):
                        if df.iloc[n, 0]:
                            first = n
                            break
                            
                    df_all = process_table(df, df_header, filename, first)
                    
                        
                    if not df_all.empty:
                        all_dfs.append(df_all)

                        print(f"Processando a página {page_num + 1} com a tabela desejada.")

        except Exception as e:
            print(f"Erro ao processar a página {page_num + 1}: {e}")

    pdf.close()

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    return final_df

### Define function to transform selected table in readable data

In [97]:
def process_table(df, df_header, filename, first):
    
    
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame()
    
    weeknumber = re.search(r'-SE(\d+)\.pdf', filename).group(1)

    df = df.iloc[first:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[:]


    df = df.loc[:, (df != '').any(axis=0)]

    df.dropna(axis=1, how='all')

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:len(df.columns)]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[0]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]

    indices_para_converter = range(1, 14)
    for idx in indices_para_converter:
        if idx < len(df.columns):
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    num_col = len(df.columns)


    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
    else:
        df1['disease'] = ''
        df1['year'] = ''


    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  


    df1['Acum_Year'] = int(df_header.iloc[1, 1].replace('\n', ' ')) - 1
    df1['week'] = weeknumber
    df1['En_Estudio'] = ''

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        'Acum.'
    ]

    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 10))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
        df2['year'] = df_header.iloc[1, 5].replace('\n', ' ')
    else:
        df2['disease'] = ''
        df2['year'] = ''


    df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)

    df2['Acum_Year'] = int(df_header.iloc[1, 5].replace('\n', ' ')) - 1

    df2['week'] = weeknumber

    column_order = [0, 2, 3, 4, 8, 5, 9, 6, 7, 10, 1]

    df2 = df2.iloc[:, column_order]

    novos_nomes = [
        df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
        df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
        df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
        df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
        'MF',
        'Acum.'
    ]

    df2.columns = novos_nomes + list(df2.columns[6:10]) + ['En_Estudio']




    df3 = df.iloc[:, [0] + list(range(10,len(df.columns)))].copy()
    if len(df_header) > 0 and len(df_header.columns) > 5:
        df3['disease'] = df_header.iloc[0, 10].replace('\n', ' ')
        df3['year'] = df_header.iloc[1, 10].replace('\n', ' ')
    else:
        df3['disease'] = ''
        df3['year'] = ''


    df3['MF'] = df3.get('Coluna12', 0) + df3.get('Coluna13', 0)


    df3['Acum_Year'] = int(df_header.iloc[1, 10].replace('\n', ' ')) - 1

    df3['week'] = weeknumber

    df3['En_Estudio'] = ''

    novos_nomes = [
        'ENTIDAD FEDERATIVA',
        'Sem.',
        'M',
        'F',
        'Acum.'
    ]

    df3.columns = novos_nomes + list(df3.columns[5:])

    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]

    df3 = df3.iloc[:, column_order]
    
    df_all = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
    

    return df_all

### Process selected year/disease and save into selected folder

In [99]:

source_path = '/home/pirata/Documents/projects/epidemic_database/files/2016/semanal'
sink_path = '/home/pirata/Documents/projects/epidemic_database/bases/2016/teste'
desired_disease = 'Dengue'
process_all_pdfs_in_folder(source_path, sink_path, desired_disease)

Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE8.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE25.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE51.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE19.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE21.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE46.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 35 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE30.pdf


...

Processando a página 35 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE12.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE32.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE10.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE11.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE3.pdf
Processando a página 27 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE52.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2016/semanal/BOL-EPID-2016-SE15.pdf
Processando a página 31 com a tabela desejada.
Salvando
Arquivo salvo: semanalDengue.csv


### Test function/ analysis

In [75]:
# filename = 'BOL-EPID-2016-SE30.pdf'

# pdf_path = '/home/pirata/Documents/projects/epidemic_database/files/2016_teste/BOL-EPID-2016-SE30.pdf'

# page_num = 30

# pdf = pdfplumber.open(pdf_path)

# tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

# if tables:
#     dfx = tables[0].df
#     page = pdf.pages[page_num]
#     table = page.extract_table()
#     df_header = pd.DataFrame(table)

In [76]:
# first = None

# for n in range(4, 10):
#     if dfx.iloc[n, 0]:
#         first = n
#         break
        
        
# print(n)

7


In [96]:
# weeknumber = re.search(r'-SE(\d+)\.pdf', filename).group(1)

# df = dfx.iloc[7:].reset_index(drop=True)
# df.columns = df.iloc[0]
# df = df[:]

# # df.replace('', np.nan, inplace=True)

# df = df.loc[:, (df != '').any(axis=0)]

# df.dropna(axis=1, how='all')

# col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
# df.columns = col_names

# for col in col_names[1:len(df.columns)]:
#     df[col] = df[col].str.replace(' ', '')

# first_col_name = df.columns[0]
# df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]

# # if len(dfx.columns) == 16:
    
# #     df = df.drop(['Coluna13','Coluna15'],axis = 1)
    
# # else:
    
# #     df = df.drop(['Coluna14'],axis = 1)


# indices_para_converter = range(1, 14)
# for idx in indices_para_converter:
#     if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
#         df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)
        
# num_col = len(df.columns)
        

# df1 = df.iloc[:, 0:5].copy()
# if len(df_header) > 1 and len(df_header.columns) > 1:
#     df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
#     df1['year'] = df_header.iloc[1, 1].replace('\n', ' ')
# else:
#     df1['disease'] = ''
#     df1['year'] = ''
    
    
# df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  


# df1['Acum_Year'] = int(df_header.iloc[1, 1].replace('\n', ' ')) - 1
# df1['week'] = weeknumber
# df1['En_Estudio'] = ''

# novos_nomes = [
#     df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
#     df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
#     df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
#     df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
#     'Acum.'
# ]

# df1.columns = novos_nomes + list(df1.columns[5:])
# column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
# df1 = df1.iloc[:, column_order]

# df2 = df.iloc[:, [0] + list(range(5, 10))].copy()
# if len(df_header) > 0 and len(df_header.columns) > 5:
#     df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
#     df2['year'] = df_header.iloc[1, 5].replace('\n', ' ')
# else:
#     df2['disease'] = ''
#     df2['year'] = ''
    
    
# df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)

# df2['Acum_Year'] = int(df_header.iloc[1, 5].replace('\n', ' ')) - 1

# df2['week'] = weeknumber

# column_order = [0, 2, 3, 4, 8, 5, 9, 6, 7, 10, 1]

# df2 = df2.iloc[:, column_order]

# novos_nomes = [
#     df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
#     df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
#     df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
#     df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
#     'MF',
#     'Acum.'
# ]

# df2.columns = novos_nomes + list(df2.columns[6:10]) + ['En_Estudio']




# df3 = df.iloc[:, [0] + list(range(10,len(df.columns)))].copy()
# if len(df_header) > 0 and len(df_header.columns) > 5:
#     df3['disease'] = df_header.iloc[0, 10].replace('\n', ' ')
#     df3['year'] = df_header.iloc[1, 10].replace('\n', ' ')
# else:
#     df3['disease'] = ''
#     df3['year'] = ''
    
# # if len(dfx.columns) == 16:
    
# #     df3['MF'] = df3.get('Coluna12', 0) + df3.get('Coluna14', 0)
    
# # else:
    
# #     df3['MF'] = df3.get('Coluna12', 0) + df3.get('Coluna13', 0)
    
    
# df3['MF'] = df3.get('Coluna12', 0) + df3.get('Coluna13', 0)
    
    
# df3['Acum_Year'] = int(df_header.iloc[1, 10].replace('\n', ' ')) - 1

# df3['week'] = weeknumber

# df3['En_Estudio'] = ''

# novos_nomes = [
#     'ENTIDAD FEDERATIVA',
#     'Sem.',
#     'M',
#     'F',
#     'Acum.'
# ]

# # novos_nomes = [
# #     df_header.iloc[0, 0].replace('\n', ' ') if len(df_header) > 0 and len(df_header.columns) > 0 else '',
# #     df_header.iloc[3, 1].replace('\n', ' ') if len(df_header) > 2 and len(df_header.columns) > 1 else '',
# #     df_header.iloc[4, 2].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 2 else '',
# #     df_header.iloc[4, 3].replace('\n', ' ') if len(df_header) > 3 and len(df_header.columns) > 3 else '',
# #     'Acum.'
# # ]

# df3.columns = novos_nomes + list(df3.columns[5:])

# column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]

# df3 = df3.iloc[:, column_order]


# df3

,ENTIDAD FEDERATIVA,Sem.,M,F,MF,Acum.,Acum_Year,disease,year,week,En_Estudio
0,Aguascalientes,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
1,Baja California,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
2,Baja California Sur,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
3,Campeche,0,2,12,14,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
4,Coahuila,0,0,1,1,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
5,Colima,6,4,36,40,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
6,Chiapas,21,54,400,454,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
7,Chihuahua,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
8,Distrito Federal,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,
9,Durango,0,0,0,0,0,2015,§Infección por Virus Zika CIE-10ª REV. U06.9,2016,30,


In [58]:
# len(df.columns)

14

In [7]:
# display(re.search(r'-SE(\d+)\.pdf', filename).group(1))

'51'

In [8]:
# desired_disease = 'Dengue'

# desired_disease in df_header.iloc[0, 1]

True

In [62]:
# len(dfx.columns)

16

In [5]:
# df_header

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,ENTIDAD\nFEDERATIVA,§Dengue no Grave\nCIE-10ª REV.\nA90,None,None,None,§Dengue con Signos de Alarma y\nDengueGrave\nC...,None,None,None,None,§Infección por Virus Zika\nCIE-10ª REV.\nU06.9,None,None,None
1,None,2016,None,None,2015,2016,None,None,None,2015,2016,None,None,2015
2,None,Confirmados,None,None,Confirmados\nAcum.,En\nEstudio\nAcum.,Confirmados,None,None,Confirmados\nAcum.,Confirmados,None,None,Confirma\ndos Acum.
3,None,Sem.,Acum.,None,None,None,Sem.,Acum.,None,None,Sem.,Acum.,None,None
4,None,None,M,F,None,None,None,M,F,None,None,M,F,None
5,Aguascalientes -\nBaja California 1\nBaja Cali...,None,- -\n7 1\n74 92\n26 44\n295 456\n124 165\n186 ...,None,22\n203\n434\n375\n65\n1 071\n1 114\n18\n-\n12...,-\n1\n37\n66\n13\n86\n63\n-\n-\n2\n1\n486\n3\n...,-\n-\n-\n-\n1\n-\n-\n-\n-\n-\n-\n3\n-\n6\n-\n2...,- -\n- -\n5 2\n10 9\n10 11\n44 46\n125 144\n- ...,None,2\n-\n3\n83\n4\n105\n454\n2\n-\n-\n-\n521\n3\n...,-\n-\n1\n-\n-\n1\n-\n-\n-\n-\n-\n-\n-\n1\n-\n-...,- 1\n- -\n3 20\n10 76\n21 37\n25 251\n98 661\n...,None,-\n-\n-\n-\n-\n-\n1\n-\n-\n-\n-\n-\n-\n-\n-\n-...
6,TOTAL 90,None,5 506 8 584,None,21 112,2 712,51,1 755 1 916,None,5 389,48,1 059 6 401,None,2


In [78]:
# dfx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,FEDERATIVA,,,,,,,,,,,,,,
1,,,2016,,2015,,2016,,,2015,,2016,,2015,
2,,,Confirmados,,,En,,Confirmados,,,,Confirmados,,,
3,,,,,Confirmados,,,,,Confirmados,,,,Confirmados,
4,,,Acum.,,,Estudio,,Acum.,,,,Acum.,,,
5,,Sem.,,,Acum.,,Sem.,,,Acum.,Sem.,,,Acum.,
6,,,M,F,,Acum.,,M,F,,,M,F,,
7,Aguascalientes,-,-,-,-,-,-,-,-,-,-,-,-,,-
8,Baja California,-,2,-,4,1,-,-,-,-,-,-,-,,-
9,Baja California Sur,-,29,24,55,4,-,-,1,3,-,-,-,,-
